In [1]:
import pandas as pd
import os
import numpy as np
import re
import emoji
from textblob import TextBlob
import nltk
from langdetect import detect
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report, confusion_matrix
nltk.download('vader_lexicon')
pd.set_option('display.max_colwidth', -1)
from scipy.sparse import coo_matrix, hstack
from scipy.sparse import csr_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
<ipython-input-1-2bbd596b2b2d>:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
sentiment_df = pd.read_csv(r"C:\Users\luoyu\Desktop\ise540\project\full_tweet_sample.csv")
sample_again = pd.read_csv(r"C:\Users\luoyu\Desktop\ise540\project\sample_again.csv")
sample_again = sample_again.rename(columns={'full_txt':'full_text'})
sentiment_df.drop(columns=['tweet_id'])
sample_again.drop(columns=['groupcount'])
sentiment_df = pd.concat([sentiment_df,sample_again])

In [3]:
list_of_txt = list(sentiment_df["full_text"])
true_sent = list(sentiment_df["sentiment"])
sid = SentimentIntensityAnalyzer()

In [4]:
true_sent = [0 if x==1 else x for x in true_sent]
true_sent = [0 if x==10 else x for x in true_sent]
#true_sent = [1 if x==-1 else x for x in true_sent]

# Train Clssifier

In [5]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

In [6]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(list_of_txt, true_sent)

In [7]:
len(train_x),len(valid_x)

(1500, 500)

### Adding sentiment  analysis feature with textblob and vader

In [91]:
def get_sentiment(text_list,method):
    textblob_vader = []
#     vader_list = []
    for each_tweet in text_list:
        blob=TextBlob(each_tweet)  #error: should be str not class<'float'>
        textblob_senti=blob.sentiment.polarity
        vader_senti=sid.polarity_scores(each_tweet)
        if method == "vader":
            textblob_vader.append(vader_senti['compound'])
        elif method == "textblob":
            textblob_vader.append(textblob_senti)
        
    return textblob_vader

In [92]:
train_textblob = get_sentiment(train_x,"textblob")
train_vader = get_sentiment(train_x,"vader")
valid_textblob = get_sentiment(valid_x,"textblob")
valid_vader = get_sentiment(valid_x,"vader")

### Keyword extraction

In [10]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,4), max_features=6000)
tfidf_vect_ngram.fit(list_of_txt)
xtrain_tfidf_ngram = tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_vect_ngram.transform(valid_x)

In [9]:

#coo_matrix([[x] for x in train_textblob])
xtrain_tfidf_ngram = csr_matrix(hstack([xtrain_tfidf_ngram,coo_matrix([[100*(x+1)] for x in train_textblob])]))
xtrain_tfidf_ngram = csr_matrix(hstack([xtrain_tfidf_ngram,coo_matrix([[100*(x+1)] for x in train_vader])]))
xvalid_tfidf_ngram = csr_matrix(hstack([xvalid_tfidf_ngram,coo_matrix([[100*(x+1)] for x in valid_textblob])]))
xvalid_tfidf_ngram = csr_matrix(hstack([xvalid_tfidf_ngram,coo_matrix([[100*(x+1)] for x in valid_vader])]))

NameError: name 'train_textblob' is not defined

In [401]:
xtrain_tfidf_ngram

<1500x6000 sparse matrix of type '<class 'numpy.float64'>'
	with 28614 stored elements in Compressed Sparse Row format>

### Embedding

In [18]:
import gensim
import gensim.downloader
nltk.download('vader_lexicon')
#split sentence to word tokens
tokens = [nltk.word_tokenize(sentences) for sentences in list_of_txt]
model = gensim.models.Word2Vec(tokens, min_count=1, size=200) 
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
<ipython-input-18-dbaae962a708>:7: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  w2v = dict(zip(model.wv.index2word, model.wv.syn0))


In [19]:
glove_vectors = gensim.downloader.load('glove-twitter-200')
glove_vectors_w2v = dict(zip(glove_vectors.wv.index2word, glove_vectors.wv.syn0))

<ipython-input-19-71df2232aaf5>:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  glove_vectors_w2v = dict(zip(glove_vectors.wv.index2word, glove_vectors.wv.syn0))
<ipython-input-19-71df2232aaf5>:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  glove_vectors_w2v = dict(zip(glove_vectors.wv.index2word, glove_vectors.wv.syn0))


In [20]:
w2v.update(glove_vectors_w2v)

In [21]:
Tfidf_model = TfidfVectorizer()
Tfidf_model.fit(list_of_txt)
tf_idf_dict = dict(zip(Tfidf_model.get_feature_names(), list(Tfidf_model.idf_)))

In [22]:
def embedding(list_of_txt):
    Tfidf_model = TfidfVectorizer()
    Tfidf_model.fit(list_of_txt)
    tf_idf_dict = dict(zip(Tfidf_model.get_feature_names(), list(Tfidf_model.idf_)))
    documents = []
    for count,tweet in enumerate(list_of_txt):
    #word_vectors = []
        weight_sum = 0
        for word in tokens[count]: # or your logic for separating tokens
            sent_vec = np.zeros(200)
            if word in tf_idf_dict:
                tf_idf_score = tf_idf_dict[word]
                sent_vec += w2v[word]*tf_idf_score
                weight_sum += tf_idf_score
                #word_vectors.append(sent_vec)
        if weight_sum != 0:
            sent_vec /= weight_sum
        documents.append(sent_vec)
    return documents 
        #documents.append(np.concatenate(word_vectors))

In [23]:
documents = embedding(list_of_txt)
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(csr_matrix(documents), true_sent)

In [24]:
def splitandtrain(documents,true_sent):
    train_x, valid_x, train_y, valid_y = model_selection.train_test_split(csr_matrix(documents), true_sent)
    report_list = train_classifier(train_x, train_y, valid_x,valid_y)
    return report_list
reports = []
for i in range(5):
    reports.append(splitandtrain(documents,true_sent))

------------------The report of KNN --------------------
              precision    recall  f1-score   support

          -1       0.43      0.24      0.31       103
           0       0.82      0.92      0.87       397

    accuracy                           0.78       500
   macro avg       0.63      0.58      0.59       500
weighted avg       0.74      0.78      0.75       500

    -1    0
-1  25  78 
 0  33  364
------------------The report of DT --------------------
              precision    recall  f1-score   support

          -1       0.34      0.37      0.35       103
           0       0.83      0.81      0.82       397

    accuracy                           0.72       500
   macro avg       0.59      0.59      0.59       500
weighted avg       0.73      0.72      0.73       500

    -1    0
-1  38  65 
 0  74  323
------------------The report of SV --------------------
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       

------------------The report of XGB --------------------
              precision    recall  f1-score   support

          -1       0.40      0.02      0.03       121
           0       0.76      0.99      0.86       379

    accuracy                           0.76       500
   macro avg       0.58      0.50      0.45       500
weighted avg       0.67      0.76      0.66       500

    -1    0
-1  2   119
 0  3   376
------------------The report of KNN --------------------
              precision    recall  f1-score   support

          -1       0.46      0.23      0.31       115
           0       0.80      0.92      0.85       385

    accuracy                           0.76       500
   macro avg       0.63      0.58      0.58       500
weighted avg       0.72      0.76      0.73       500

    -1    0
-1  27  88 
 0  32  353
------------------The report of DT --------------------
              precision    recall  f1-score   support

          -1       0.35      0.38      0.37      

In [25]:
train_classifier(train_x, train_y, valid_x)

TypeError: train_classifier() missing 1 required positional argument: 'valid_y'

### Model selection

In [13]:
MN = naive_bayes.MultinomialNB()
KNN = KNeighborsClassifier()
LR = linear_model.LogisticRegression()
DT = tree.DecisionTreeClassifier()
SV = SVC(kernel='rbf', probability=True)
GB = GradientBoostingClassifier(random_state=0)
RF = RandomForestClassifier()
XGB = xgb_model = xgb.XGBClassifier(base_score=0.5, booster='gblinear', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=1.0, gamma=5, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.02, max_delta_step=None, max_depth=5,
              min_child_weight=5, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=None,
              objective='binary:hinge', random_state=0, reg_alpha=0,
              reg_lambda=0, scale_pos_weight=None, subsample=0.6,
              tree_method=None, validate_parameters=1, verbosity=None)

test_classifiers = [ KNN, DT, SV, GB, RF, XGB]
test_classifiers_flag = ['KNN','DT', 'SV','GB','RF', 'XGB']

In [12]:
test_classifiers = [DT]
test_classifiers_flag = ['DT']

In [14]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,valid_y):
    classifier.fit(feature_vector_train, label)
    predictions = classifier.predict(feature_vector_valid)
    labels = np.unique(true_sent)
    a =confusion_matrix(valid_y,predictions,labels = labels)
    conf = pd.DataFrame(a, index=labels, columns=labels)
    report = classification_report(valid_y,predictions,labels = labels)
    
    return metrics.f1_score(valid_y,predictions, average='weighted'),conf,report
# metrics.accuracy_score(valid_y,predictions),

In [15]:
def train_classifier(X_train, y_train, X_valid,valid_y):
    report_list=[]
    for classifier, flag in zip(test_classifiers, test_classifiers_flag):
        f1,conf,report=train_model(classifier, X_train, y_train, X_valid,valid_y)
        print('------------------The report of',flag,'--------------------')
        print(report)
        print(conf)
        report_list.append((flag,report))
    return report_list

In [16]:
def splitandtrain(list_of_txt,true_sent):
    train_x, valid_x, train_y, valid_y = model_selection.train_test_split(list_of_txt, true_sent)
    tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,4), max_features=6000)
    tfidf_vect_ngram.fit(list_of_txt)
    xtrain_tfidf_ngram = tfidf_vect_ngram.transform(train_x)
    xvalid_tfidf_ngram = tfidf_vect_ngram.transform(valid_x)
    report_list = train_classifier(xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram,valid_y)
    return report_list

In [17]:
splitandtrain(list_of_txt,true_sent)

------------------The report of KNN --------------------
              precision    recall  f1-score   support

          -1       0.45      0.19      0.27       103
           0       0.82      0.94      0.87       397

    accuracy                           0.79       500
   macro avg       0.64      0.57      0.57       500
weighted avg       0.74      0.79      0.75       500

    -1    0
-1  20  83 
 0  24  373
------------------The report of DT --------------------
              precision    recall  f1-score   support

          -1       0.43      0.42      0.42       103
           0       0.85      0.85      0.85       397

    accuracy                           0.76       500
   macro avg       0.64      0.64      0.64       500
weighted avg       0.76      0.76      0.76       500

    -1    0
-1  43  60 
 0  58  339
------------------The report of SV --------------------
              precision    recall  f1-score   support

          -1       0.83      0.18      0.30       

[('KNN',
  '              precision    recall  f1-score   support\n\n          -1       0.45      0.19      0.27       103\n           0       0.82      0.94      0.87       397\n\n    accuracy                           0.79       500\n   macro avg       0.64      0.57      0.57       500\nweighted avg       0.74      0.79      0.75       500\n'),
 ('DT',
  '              precision    recall  f1-score   support\n\n          -1       0.43      0.42      0.42       103\n           0       0.85      0.85      0.85       397\n\n    accuracy                           0.76       500\n   macro avg       0.64      0.64      0.64       500\nweighted avg       0.76      0.76      0.76       500\n'),
 ('SV',
  '              precision    recall  f1-score   support\n\n          -1       0.83      0.18      0.30       103\n           0       0.82      0.99      0.90       397\n\n    accuracy                           0.82       500\n   macro avg       0.82      0.59      0.60       500\nweighted av

In [213]:
reports = []
for i in range(5):
    reports.append(splitandtrain(list_of_txt,true_sent))

------------------The report of DT --------------------
              precision    recall  f1-score   support

          -1       0.43      0.47      0.45       107
           0       0.85      0.83      0.84       393

    accuracy                           0.76       500
   macro avg       0.64      0.65      0.65       500
weighted avg       0.76      0.76      0.76       500

    -1    0
-1  50  57 
 0  65  328
------------------The report of DT --------------------
              precision    recall  f1-score   support

          -1       0.36      0.31      0.33       100
           0       0.83      0.86      0.85       400

    accuracy                           0.75       500
   macro avg       0.60      0.59      0.59       500
weighted avg       0.74      0.75      0.74       500

    -1    0
-1  31  69 
 0  55  345
------------------The report of DT --------------------
              precision    recall  f1-score   support

          -1       0.43      0.40      0.41       1

### Select model parameter

In [449]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import StratifiedKFold
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [4, 5, 6],
        'objective': ['binary:logistic','binary:hinge'],
        'booster' : ['gbtree','gblinear','dart']
        }
XGB = xgb.XGBClassifier(learning_rate=0.02, objective='binary:logistic',silent=True)

In [453]:
folds = 5
param_comb = 50

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(XGB, param_distributions=params, n_iter=param_comb, scoring='recall', n_jobs=4, cv=skf.split(train_x, train_y), verbose=3, random_state=1001 )

random_search.fit(train_x, train_y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   28.8s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:  3.9min finished


[22:46:48] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x000001CBAA65EE40>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.02,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, mis...
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                           verbosity=None),
                   n_iter=50, n_jobs=4,
                   

In [264]:
parameter = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(tree.DecisionTreeClassifier(random_state=42), 
                              param_grid=parameter, verbose=1, cv=3,scoring='recall')
grid_search_cv.fit(xtrain_tfidf_ngram, train_y)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 882 out of 882 | elapsed:  1.3min finished


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             scoring='recall', verbose=1)

In [265]:
grid_search_cv.best_estimator_
grid_search_cv.best_score_

0.3522012578616352

In [455]:
print(random_search.best_estimator_)
print(random_search.best_score_)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=0.5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=6,
              min_child_weight=10, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:hinge', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, silent=True, subsample=0.6,
              tree_method='exact', validate_parameters=1, verbosity=None)
0.1681592039800995


## Predict 

In [133]:
data=pd.read_pickle(r"C:\Users\luoyu\Desktop\ise540\project\cleaned.pkl")

In [142]:
data['full_txt'].isna().sum()

0

In [135]:
data['full_txt']=data['og_tweet_txt']+data['tweet_txt']
full_txt=data['full_txt'].values.tolist()

### TFIDF

In [137]:
xtest_tfidf_ngram = tfidf_vect_ngram.transform(full_txt)

In [138]:
DT.fit(tfidf_vect_ngram.transform(list_of_txt), true_sent)
predictions = DT.predict(xtest_tfidf_ngram)

### Embedding

In [122]:
tokens = [nltk.word_tokenize(sentences) for sentences in full_txt]
model = gensim.models.Word2Vec(tokens, min_count=1, size=200) 
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

<ipython-input-122-09745ea311b4>:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  w2v = dict(zip(model.wv.index2word, model.wv.syn0))


In [123]:
w2v.update(glove_vectors_w2v)
Tfidf_model = TfidfVectorizer()
Tfidf_model.fit(full_txt)
tf_idf_dict = dict(zip(Tfidf_model.get_feature_names(), list(Tfidf_model.idf_)))

In [125]:
dembed_train = embedding(list_of_txt)
DT.fit(csr_matrix(dembed_train), true_sent)

5753


DecisionTreeClassifier()

In [127]:
embed_full = embedding(full_txt)
predictions = DT.predict(csr_matrix(embed_full))

In [121]:
len(full_txt)

357474

In [139]:
pred=predictions.tolist()

In [140]:
counts = { }          
for x in pred:                      
    if  x in counts:
        counts[x] += 1
    else:
        counts[x] = 1
print(counts)

{0: 285180, -1: 72294}


In [19]:
# dic=dict(zip(full_txt,pred))

In [141]:
data['predict']=predictions
data.to_csv('prediction_TFIDF_DT.csv')